In [1]:
from collections import namedtuple
import os
os.chdir("../")

In [2]:
# Defining the entity(Structure) for Data Ingestion Config Object which will serve as input to Data Ingestion Component

DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [3]:
from src.DeepLearningModel.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.DeepLearningModel.utils import read_yaml, create_directories

In [4]:
CONFIG_FILE_PATH

WindowsPath('configs/config.yaml')

In [5]:
# The Configration manager is responsible for holding the methods that are used to 
# a) Create nessesary folder structure in root for each component's config object
# b) Assigning values to Configs which are defined in entity by reading input from config.yaml and returning config object 

class ConfigurationManager:
    # The constructor will read config.yaml and params.yaml and will ceate nessesary folder for artifacts
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        print(config_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        print(params_filepath)
        create_directories([self.config.artifacts_root])

    # The get_data_ingestion_config function will 
    # a) Read config values from config.yaml
    # b) Will create folder in artifact for data ingestion config object
    # c) Assign the values to DataIngestionConfig object and return it
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
# The component takes config object as input
# The component holds all the definations for fuctions associated with execution of activities in a block of pipeline

import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    # The constructor will associate config object with component
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Function for downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    # Function for retaining only those files which are .jpeg and are in folder cats or dogs
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    # Function for extracting images one by one if image doesn't exist and if its size is not zero 
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    # Reading one by one images from zip and sending it for extraction
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist(    )
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [7]:
try:
    os.chdir("c:\\Users\\AK\\DL_CLASSIFIER")
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

configs\config.yaml
params.yaml
